# Fine-tuning du modèle GPT-2 sur des articles scientifiques

Dans ce notebook, nous nous sommes focalisés sur l'entrainement par fine-tuning du modèle GPT-2, sur un jeu de données d'articles scientifiques scientific_papers (dataset Tensorflow).

L'objectif est de pouvoir spécialiser un modèle générique (modèle GPT-2 base) sur un domaine spécifique via un entrainement sur un jeu de données fourni (dataset TensorFlow scientific_papers)

**Groupe ALOQAS**
- Aurélien ZUFIC
- Lucas AGUETAÏ
- Ony ANDRIATSAHAVOJAONA
- Quentin VERMEERSCH
- Alexandre HUYNH
- Samuel DORISMOND

**NOTE /!\ : L'entrainement s'est effectué en deux temps (TRAIN PART 1 & 2), étant donné qu'il était impossible de charger et entraîner le modèle sur le jeu de données en entier (limitations mémoire RAM). Il sera nécessaire de commenter et décommenter certaines sections de code pour l'exécution en cas d'entrainement à partir du début.**

---
# Import de packages
Importation des packages Python nécessaires pour le fine-tuning.

Nous chargeons également un package personnel datasets_scientific_paper, contenant des fonctions Python écrites pour le chargement de la dataset localement.

Celle-ci est divisée en plusieurs parties à travers plusieurs fichiers, afin de prendre en compte les performances limités de Google Colaboratory.

In [ ]:
!pip install keras_nlp -q

In [ ]:
import tensorflow as tf
import json
import os
import keras_nlp
import time
from tensorflow import keras
import numpy as np

Using TensorFlow backend


In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/package')
import datasets_scientific_paper as load_ds

Mounted at /content/drive
Mounted at /content/drive


---
# Chargement dataset train en local

Pour le fine-tuning du modèle GPT-2, nous utilisons pour l'instant seulement les articles d'entrainement (le jeu de données entier d'origine est divisée en 3 parties : train, val, test).

Le package importée précédemment nous permet d'utiliser une fonction personnalisée load_dataset pour charger dans des variables Python le jeu de données sous forme de dictionnaires et de listes.

In [ ]:
# chemin vers le jeu de données divisée localement
pathToDataset = "drive/MyDrive/chunking-dataset"

In [ ]:
# "train" afin de prendre seulement la partie du jeu de données dédié à l'entrainement du modèle
# 100 afin de prendre 100 fichiers de 10.000 articles

# === TRAIN PART 1 ===
#train_data, train_labels = load_ds.load_dataset(pathToDataset, "train", 81)

# === TRAIN PART 2 ===
train_data, train_labels = load_ds.load_dataset(pathToDataset, "train_extract_3", 41)
# en raison des limitations mémoires sur Colab
# utilisation d'un autre répertoire créé manuellement : chunking-dataset/train_extract_3
# copier manuellement les fichiers train_part-80 à train_part-119 dans ce répertoire
# afin de charger seulement les articles des fichiers 80 à 119

---
# Initialisation d'un modèle GPT-2 de base
Nous initialisons une instance du modèle GPT-2 sur lequel nous effectuons l'entrainement.

Le modèle et code utilisés sont celui proposés via Keras NLP.

Il existe plusieurs modèles préentrainées pour GPT-2, ayant un nombre de couches et de paramètres différent : celui sélectionné dans notre cas est le modèle gpt2_base_en, qui est suffisant pour notre entrainement.

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

keras.mixed_precision.set_global_policy("mixed_float16")

preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Nous définissons une fonction articles_to_text nous permettant de concaténer tous les mots des articles dans une liste Python unique.

En effet, chaque article possèdent plusieurs champs d'attributs : article_text, abstract, section_names...

Le plus important pour le fine-tuning est d'entrainer le modèle sur le contenu textuel, donc article_text. Cette dernière - après chargement via notre fonction personnalisée - est composée pour chaque article d'une liste Python constitué de sous listes python pour plusieurs portions du texte. La fonction présente reformatte cela afin de ne garder qu'une seule liste Python contenant tous les mots sur lequel l'apprentissage doit se faire.

In [ ]:
def articles_to_text(articles, text_key, n_start, n_end):
  article_list = []
  for article in articles[n_start:n_end] :
    article_text = " ".join(article.get(text_key, ""))
    article_list.append(article_text)
  return article_list

---
# Chargement des paramètres d'entrainement

Il existe plusieurs méthodes pour l'enregistrement des résultats de l'entrainement d'un modèle sur un support fichier.

Basé sur les enseignements fournis par notre enseignant M. Faye, nous avons retenus trois méthodes :
- **Fichier .keras** : enregistrement du **modèle** (architecture, couches)
- **Fichier .ckpt** : enregistrement des **paramètres / poids** mobilisés
- **Fichier .h5** : enregistrement du **modèle** et des **poids** mobilisés

Il nous a été recommandé pour le fine-tuning de préférer l'enregistrement des paramètres. Il nous suffit donc à chaque début d'initialiser le modèle GPT-2 de base et de charger l'entrainement effectué via le chargement des poids/paramètres (fonction .load_weights).

In [ ]:
checkpoint_path = "/content/drive/MyDrive/training_data_all_3/cp.ckpt"

In [ ]:
# Chargement des poids du modèle (facultatif)
# - exécuter seulement si des fichiers de checkpoints existent
gpt2_lm.load_weights(checkpoint_path)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


---
# Entrainement / Fine-tuning

C'est dans cette partie que le fine-tuning du modèle GPT-2 est réalisé.

## Principe

Le principe de ce fine-tuning est de fournir un grand nombre de mots et de phrases d'articles scientifiques dans un tableau Python qui sera fourni à une fonction .fit pour l'entrainement du modèle. Le modèle va se charger d'apprendre sur ce qui est fourni afin d'orienter la génération de texte du modèle GPT-2 vers un vocabulaire et style similaire aux articles scientifiques.

Dans ce code, si des fichiers de checkpoint sont trouvés, le programme charge les poids existants, sinon il entraine le modèle et sauvegarde de nouveaux poids.

## Entrainement sur échantillons d'articles scientifiques

Nous fournissons à la fonction .fit réalisant l'entrainement une liste Python features, auquel on donne comme liste le contenu textuel d'articles obtenus via la fonction articles_to_text, mentionnée précédemment.

Plusieurs essais ont été menés progressivement avec des échantillons de nombre croissant. Pour s'assurer que l'entrainement était faisable compte tenu des performances et ressources à notre disposition, nous avions débuté avec 1 à 10 articles.

Nous avons procédé par la suite à des nombres d'articles plus importants : 10.000 articles, 50.000, et enfin 80.000 articles, limite atteinte d'articles mobilisables dans un même entrainement.

## Callback ModelCheckpoint

Un callback est utilisé afin d'effectuer une sauvegarde des paramètres à chaque "checkpoint" (point de contrôle) en fin d'epoch. Etant donné que nous avons défini seulement 1 seul epoch, l'enregistrement des poids se fait en fin d'entrainement. <br />
Cela entraîne la création de fichiers .ckpt constituant l'entrainement du modèle, sous la forme de divers poids pour le modèle GPT-2.

## Mobilisation du GPU

Afin de permettre la réalisation de l'entrainement en évitant la saturation de la mémoire vive, il était nécessaire de mobiliser le GPU en plus de la RAM.

Nous avons donc inclus notre dans une structure Python spécifique qui nous permet de solliciter le GPU pour une partie spécifique du code.
```python
with tf.device('/device:GPU:0'):
```


In [ ]:
# Définir le chemin et le répertoire du checkpoint
checkpoint_dir = os.path.dirname(checkpoint_path)

# Créer le répertoire s'il n'existe pas
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# === TRAIN PART 1 ===
#features = articles_to_text(train_data, "article_text", 0, 80000)

# === TRAIN PART 2 ===
features = articles_to_text(train_data, "article_text", 0, len(train_data)-1)

# Créer un callback pour sauvegarder les poids du modèle
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

print("Training GPT-2 model...")

# Entraîner le modèle avec GPU et utiliser le callback de checkpoint
with tf.device('/device:GPU:0'):
    num_epochs = 1

    # Taux d'apprentissage décroissant linéairement
    learning_rate = keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=5e-5,
        decay_steps=len(features) * num_epochs,
        end_learning_rate=0.01,
    )

    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    gpt2_lm.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        weighted_metrics=["accuracy"],
    )

    # Entraîner le modèle avec le callback de checkpoint
    gpt2_lm.fit(
        x=features,
        epochs=num_epochs,
        callbacks=[cp_callback]  # Passer le callback de checkpoint à l'entraînement
    )

Training model from loaded model checkpoints...
1248/1248 [==============================] - ETA: 0s - loss: 2.9164 - accuracy: 0.4298
Epoch 1: saving model to /content/drive/MyDrive/training_data_all_3/cp.ckpt
1248/1248 [==============================] - 1624s 1s/step - loss: 2.9164 - accuracy: 0.4298


---
# Test évaluation du modèle

Après avoir effectué le fine-tuning du modèle GPT-2, nous avons mené des tests d'évaluation simples.

Des tests plus approfondis seront menés dans les prochaines phases de projet.

In [ ]:
# Charger les données de validation
val_data, val_labels = load_ds.load_dataset(pathToDataset, "val", 7)

# Rassembler le texte des articles en une seule liste
val_features = articles_to_text(val_data, "article_text", 0, len(val_data)-1) # 0-6632

In [ ]:
# Évaluer le modèle sur les données d'évaluation
with tf.device('/device:GPU:0'):
  val_loss, val_accuracy = gpt2_lm.evaluate(
      x=val_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

208/208 [==============================] - 204s 912ms/step - loss: 2.9360 - sparse_categorical_accuracy: 0.4361
Validation Loss: 2.9359724521636963
Validation Accuracy: 0.4360581338405609


In [ ]:
# Charger les données de test
test_data, test_labels = load_ds.load_dataset(pathToDataset, "test", 7)

# Rassembler le texte des articles en une seule liste
test_features = articles_to_text(test_data, "article_text", 0, len(test_data)-1) # 0-6657

In [ ]:
# Évaluer le modèle sur les données de test
with tf.device('/device:GPU:0'):
  test_loss, test_accuracy = gpt2_lm.evaluate(
      x=test_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

209/209 [==============================] - 194s 905ms/step - loss: 2.9129 - sparse_categorical_accuracy: 0.4390
Test Loss: 2.9129157066345215
Test Accuracy: 0.43897753953933716


---
# Test génération de texte à partir de prompt

Nous avons essayé de générer du texte à partir d'exemple de prompts, afin de vérifier que le modèle reste cohérent après celui, et également pour avoir un aperçu de la spécialisation de celle-ci sur les thèmatiques des articles scientifiques.

In [ ]:
# Exemple de prompts basés sur des sujets de papiers scientifiques
prompts = [
    "The impact of global warming on marine biodiversity",
    "Technological advancements in renewable energy sources",
    "Genetic factors influencing Alzheimer's disease",
    "The role of artificial intelligence in personalized medicine",
    "Quantum computing and its future implications",
    "Mechanisms of resistance to antibiotics in bacteria",
    ""
]

# Parcourir et générer des réponses pour chaque prompt
for prompt in prompts:
    start = time.time()
    output = gpt2_lm.generate(prompt, max_length=200)
    end = time.time()

    print(f"\nPrompt: {prompt}")
    print("GPT-2 output:")
    print(output)
    print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")
    print('________________________________________________')


Prompt: The impact of global warming on marine biodiversity
GPT-2 output:
The impact of global warming on marine biodiversity is well recognized [ 1 , 2 ] . the impact of climate change is expected to be significant for both human and marine ecosystems [ 3 , 4 ] . in addition , the impact of the global climate on human health has been recognized for decades [ 59 ] . in addition , the effects of climate change are also being recognized [ 10 , 1112 ] . in addition to the impact of climate change , it has been recognized in recent years that human and marine ecosystems have a complex interaction . the human ecosystem has many different types of ecosystem , and these interactions may lead to different changes in the environment [
TOTAL TIME ELAPSED: 20.92s
________________________________________________

Prompt: Technological advancements in renewable energy sources
GPT-2 output:
Technological advancements in renewable energy sources ( e.g. , wind and solar power , geothermal energy sour